<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-0.0.0.1"><span class="toc-item-num">0.0.0.1&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-1.0.0.2"><span class="toc-item-num">1.0.0.2&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-1.0.0.3"><span class="toc-item-num">1.0.0.3&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-1.0.0.4"><span class="toc-item-num">1.0.0.4&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-1.0.0.5"><span class="toc-item-num">1.0.0.5&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#-Комментарий-студента-💎" data-toc-modified-id="-Комментарий-студента-💎-3.0.0.1"><span class="toc-item-num">3.0.0.1&nbsp;&nbsp;</span> Комментарий студента 💎</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В данном проекте нам необходимо обучить модель классифицировать комментарии на позитивные и негативные. 
В нашем распоряжении имеется набор данных с разметкой о токсичности правок.

Целевое значение метрики качества *F1* не меньше 0.75. 

Весь проект можно разделить на четыре основных этапа:

1. Подготовка данных.
2. Обучение моделей. 
3. Тестирование лучшей модели.
4. Общий вывод.

Для выполнения данного проекта будет применяться *BERT*.


## Подготовка данных

In [1]:
pip install torch -q # -q убирает необязательные выводы в командах Linux

In [2]:
pip install lightgbm

In [3]:
pip install catboost

In [4]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [5]:
# загружаем необходимые библиотеки

import pandas as pd
import numpy as np
import nltk
import transformers as ppb
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix
from tqdm import notebook

import torch
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [6]:
# константы

MAX_TOKENS=510

In [7]:
data = pd.read_csv('toxic_comments.csv')
data.info()
data.head()

In [8]:
# так как создание embeddings для всего датасета может занять очень много времени, возьмем 5000 случайных строк

data = data.sample(5000, random_state=12345)
data.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'unitary/toxic-bert')

In [10]:
# загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [11]:
# токенизируем наши данные
# обрежем предложения, которые длиннее чем 510 токенов, так как максимальная длина, с которой может работать модель - 512
# 2 слота оставляем для токена начала и окончания

tokenized = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=MAX_TOKENS, truncation=True)))


In [12]:
# теперь приведём каждый вектор в соответствие с длиной самого длинного вектора
# для этого проставим нули в количестве, не достающем до длины максимального вектора

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
padded

In [13]:
# создадим attention_mask, чтобы показать, какие элементры каждого вектора важны (все ненулевые значения)
attention_mask = np.where(padded != 0, 1, 0)


In [18]:
# теперь создадим embeddings, разбивая весь датасет на небольшие батчи

batch_size = 50
embeddings = []

model.cuda()
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        
        del batch
        del attention_mask_batch
        del batch_embeddings

In [19]:
# объединим все embeddings в один датасет features
features = pd.DataFrame(np.concatenate(embeddings))

In [20]:
features.index = data.index

In [ ]:
# features = pd.read_csv('features.csv')
# features.drop('Unnamed: 0', axis=1, inplace=True)
# features

In [21]:
# разбиваем данные на обучающую и тестовую выборки

features_train, features_test, target_train, target_test = train_test_split(
    features,
    data['toxic'],
    random_state=12345
)

display(features_train.shape, features_test.shape, target_train.shape, target_test.shape)

In [22]:
(target_train == 1).mean()

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

Итак в первом этапе мы сделали следующие действия:
- Загрузили имеющиеся данные.
- Выделили 5000 случайных строк, с которыми будет работать BERT.
- Токенизировали наши твиты.
- Провели padding - сделали вектора токенов одной длины, равной самому длинному твиту.
- Создали attention_mask и разметили, какие данные в каждом векторе важны для BERT.
- Создали embeddings, используя разделение по батчам в целях экономии памяти, из которых сформировали features.
- Раздили features и target на обучающую и тестовую выборки в соотношении 3:1.

## Обучение моделей

В данном разделе нам необходимо обучить несколько моделей и выбрать из них лучшую.  

Рассмотрим модели дерева решений, случайного леса, логистической регрессии и LGBM.

In [24]:
# дерево решений

decision_tree = DecisionTreeClassifier(random_state=12345)

tree_parameters_grid = {
   'max_depth' : range(1, 16),
}

tree_grid_cv = GridSearchCV(
    decision_tree,
    tree_parameters_grid,
    cv=3,
    scoring='f1',
    verbose=3,
    error_score='raise'
)
tree_grid_cv.fit(features_train, target_train)
print(f'Наилучшее значение параметра max_depth: {tree_grid_cv.best_params_}') 
print(f'Наилучший показатель F1_score: {tree_grid_cv.best_score_}')

Дерево решений:
- max_depth=2.
- F1_score ~ 0.923.

In [26]:
# модель случайного леса

random_forest = RandomForestClassifier(random_state=12345)

forest_parameters_grid = {
    'max_depth' : range(10, 21, 2),
    'n_estimators' : range(80, 210, 40)
}

forest_grid_cv = GridSearchCV(
    random_forest,
    forest_parameters_grid,
    cv=3,
    scoring='f1',
    verbose=3,
    error_score='raise',
    n_jobs=-1
)
forest_grid_cv.fit(features_train, target_train)
print(f'Наилучшее значение параметра max_depth: {forest_grid_cv.best_params_}') 
print(f'Наилучший показатель F1_score: {forest_grid_cv.best_score_}')

Дерево решений:
- max_depth=10.
- n_estimators=200.
- F1_score ~ 0.935.

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [33]:
# логистическая регрессия 

log_regression = LogisticRegression(random_state=12345)

log_regression_grid = {
    'class_weight' : ['balanced', None],
    'penalty' : ['l1', 'l2'],
    'C' : [0.1, 0.5, 1.0],
    'solver' : ['liblinear']
}

log_regression_grid_cv = GridSearchCV(
    log_regression,
    log_regression_grid,
    cv=5,
    scoring='f1',
    verbose=3,
    error_score='raise',
    n_jobs=-1
)
log_regression_grid_cv.fit(features_train, target_train)
print(f'Наилучшее значение параметра max_depth: {log_regression_grid_cv.best_params_}') 
print(f'Наилучший показатель F1_score: {log_regression_grid_cv.best_score_}')

Логистическая регрессия:
- С - 0.1.
- class_weight - None.
- penalty - l2.
- F1_score ~ 0.937.

In [34]:
# Теперь переходим к градиентному бустингу с помощью LGBM

lgbm_model = LGBMClassifier(random_state=12345)

lgbm_parameters_grid = {
    'max_depth' : range(10, 21, 2),
    'num_leaves' : [31],
    'n_estimators' : range(80, 170, 40),
}

lgbm_grid_cv = GridSearchCV(
    lgbm_model,
    lgbm_parameters_grid,
    cv=3,
    scoring='f1',
    verbose=3,
    error_score='raise',
    n_jobs=-1
)
lgbm_grid_cv.fit(features_train, target_train)
print(f'Наилучшие значения параметров: {lgbm_grid_cv.best_params_}') 
print(f'Наилучший показатель F1_score: {lgbm_grid_cv.best_score_}')

Дерево решений:
- max_depth=10.
- n_estimators=160.
- F1_score ~ 0.936.

По итогам построения моделей лучшей оказалась модель логистической регрессии.  
В следующем этапе работы мы поверим её на тестовой выборке.

## Тестирование лучшей модели

In [36]:
log_regression = LogisticRegression(
    solver='liblinear',
    penalty='l2',
    C=0.1,
    random_state=12345
)

log_regression.fit(features_train, target_train)

pred = log_regression.predict(features_test)

f1_score(target_test, pred)


Наша выбранная модель показала на тестовой выборке результат метрики F1 ~ 0.952.

Сравним этот результат с наивным столбцом, где все значения = 1.

Логистическая регрессия:
- С - 0.1.
- class_weight - None.
- penalty - l2.
- F1_score ~ 0.937.

In [ ]:
ones_series = pd.Series(1, index=range(len(target_test)))

f1_score(target_test, ones_series)


## Общий вывод

Итого в данном проекте мы выполнили следующие этапы.

**1 этап**

- Загрузили имеющиеся данные.
- Выделили 5000 случайных строк, с которыми будет работать BERT.
- Токенизировали наши твиты.
- Провели padding - сделали вектора токенов одной длины, равной самому длинному твиту.
- Создали attention_mask и разметили, какие данные в каждом векторе важны для BERT.
- Создали embeddings, используя разделение по батчам в целях экономии памяти, из которых сформировали features.
- Раздили features и target на обучающую и тестовую выборки в соотношении 3:1.


**2-3 этапы**

- Обучили несколько моделей, включая дерево решений, случайный лес, логистическую регрессию, LGBM.
- Подобрали гиперпараметры и провели кросс-валидацию.
- Выбрали из лучшую модель и проверили её на тестовой выборке.
- Итоговая метрика F1 получилась равна 0.71.